In [ ]:
!pip install gradio
!pip install openai requests beautifulsoup4
!pip install langgraph langchain openai gradio beautifulsoup4
!pip install tavily-python
!from tavily import TavilyClient
!pip install langchain tavily-python openai
!pip install python-dotenv
from tavily import TavilyClient
!pip install langchain_community
!pip install --upgrade langchain langchain_community

from langchain.llms import HuggingFaceHub
!pip install --upgrade huggingface_hub
from huggingface_hub import InferenceClient
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool, AgentType

import os
from dotenv import load_dotenv
import openai
import requests
import gradio as gr
from bs4 import BeautifulSoup
from google.colab import files
uploaded = files.upload()

# Конфигурация API NewsAPI
NEWS_API_URL = "https://newsapi.org/v2/everything"

class Config:
    @staticmethod
    def setup():
        # Загружаем переменные из .env файла
        load_dotenv()
        token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
        # Настройка переменных окружения для LangChain, Tavily и OpenAI
        os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY", "")
        os.environ["LANGCHAIN_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT", "")
        os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGSMITH_PROJECT", "")
        os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY", "")
        os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "")
        os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN", "")

        if os.getenv("LANGSMITH_TRACING", "false").lower() == "true":
            os.environ["LANGCHAIN_TRACING_V2"] = "true"

# Запускаем настройку  # Инициализация клиента

client = InferenceClient(token=os.getenv("HUGGINGFACEHUB_API_TOKEN"))

# Вызываем модель
response = client.text_generation(
    "Hi! Tell me something interesting..",
    model="mistralai/Mistral-7B-Instruct-v0.1",
    temperature=0.7,
    max_new_tokens=100,
)

print(response)

llm = HuggingFaceHub(repo_id="google/flan-t5-base", model_kwargs={"temperature":0, "max_length":512})

print(llm("Hi! Tell me something interesting.."))

# 🧠 Инструмент Tavily
def tavily_search_tool(query: str) -> str:
    tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
    results = tavily.search(query=query, search_depth="basic")

    summaries = []
    for res in results['results'][:3]:
        try:
            html = requests.get(res['url']).content
            soup = BeautifulSoup(html, 'html.parser')
            text = ' '.join([p.get_text() for p in soup.find_all('p')])
            summaries.append(text[:500])
        except Exception as e:
            summaries.append(f"[Fehler {res['url']}: {e}]")
    return '\n\n'.join(summaries)


# ✅ Регистрируем инструмент LangChain
tools = [
    Tool(
        name="Tavily Web Search",
        func=tavily_search_tool,
        description=""Searches for up-to-date market information on the Internet at the user's request""
    )
]

# 🤖 Инициализация агента
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 🎯 Обработчик запроса
def agent_handler(user_input):
    try:
        response = agent.run(user_input)
        return response
    except Exception as e:
        return f"❌ Agent Error: {str(e)}"

# 🎛 Интерфейс Gradio
gr.Interface(
    fn=agent_handler,
    inputs=gr.Textbox(label="Your query (eg: What news is impacting Google today?))"),
    outputs=gr.Textbox(label="Agent's response"),
    title="🧠 Web search agent based onTavily + OpenAI",
    description="This agent searches for fresh news and information using Tavily and analyzes it using GPT."
).launch()

from langgraph.prebuilt import create_react_agent

Web_Search_Agent = create_react_agent(llm, [retrieve])
